# GA & PSO Comparison

In [ ]:
# --- Import Required Libraries ---
import pandas as pd
from scipy.stats import wilcoxon, ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys
sns.set_theme(style="whitegrid")

# --- Set Up Paths ---
ROOT_PATH = os.path.abspath(os.path.join(os.getcwd(), '..'))
if ROOT_PATH not in sys.path:
    sys.path.insert(0, ROOT_PATH)

# --- Import Custom Libraries ---
from pop.util.fine_tuning import get_results_path

In [ ]:
final_results_df = pd.read_csv(get_results_path("final_fine_tuning_results.csv"))
ga_sharpe = final_results_df[final_results_df["algorithm"] == "GA"]["sharpe_ratio"]
pso_sharpe = final_results_df[final_results_df["algorithm"] == "PSO"]["sharpe_ratio"]

t_stat, p_value = ttest_ind(ga_sharpe, pso_sharpe, equal_var=False)
print(f"T-test for Sharpe Ratio: t={t_stat:.3f}, p={p_value:.3g}")

In [ ]:
print(final_results_df.groupby(["algorithm", "quality"])[["sharpe_ratio", "annual_return", "runtime"]].describe())

## Load GA & PSO Results

In [ ]:
# # Paths to results
# GA_RESULTS_PATH = "experiments/results/ga/"
# PSO_RESULTS_PATH = "experiments/results/pso/"

# # Load results
# ga_results = pd.read_csv(GA_RESULTS_PATH)
# pso_results = pd.read_csv(PSO_RESULTS_PATH)

# # Add algorithm labels
# ga_results["algorithm"] = "GA"
# pso_results["algorithm"] = "PSO"

# # Combine results
# combined_results = pd.concat([ga_results, pso_results], ignore_index=True)

## Plot Boxplots for Fitness & Evaluations

In [ ]:
# # Plot fitness boxplots
# plt.figure(figsize=(12, 8))
# sns.boxplot(x="num_companies", y="sharpe_ratio", hue="algorithm", data=combined_results, palette={"GA": "blue", "PSO": "orange"})
# plt.title("Fitness Comparison by Portfolio Size")
# plt.xlabel("Number of Assets")
# plt.ylabel("Sharpe Ratio")
# plt.legend(title="Algorithm")
# plt.show()

# # Plot evaluations boxplots
# plt.figure(figsize=(12, 8))
# sns.boxplot(x="num_companies", y="n_evaluations", hue="algorithm", data=combined_results, palette={"GA": "blue", "PSO": "orange"})
# plt.title("Evaluations Comparison by Portfolio Size")
# plt.xlabel("Number of Assets")
# plt.ylabel("Number of Evaluations")
# plt.legend(title="Algorithm")
# plt.show()

## Wilcoxon Signed-Rank Test for Pairwise Comparison

In [ ]:
# alpha = 0.05

# # Iterate over unique portfolio sizes
# for num_companies in combined_results["num_companies"].unique():
#     print(f"\nPortfolio Size: {num_companies}")

#     # Filter results for the current portfolio size
#     ga_data = combined_results[(combined_results["num_companies"] == num_companies) & (combined_results["algorithm"] == "GA")]
#     pso_data = combined_results[(combined_results["num_companies"] == num_companies) & (combined_results["algorithm"] == "PSO")]

#     # Perform Wilcoxon test for fitness
#     stat, p_value = wilcoxon(ga_data["sharpe_ratio"], pso_data["sharpe_ratio"])
#     print(f"Fitness - GA Mean: {ga_data['sharpe_ratio'].mean():.4f}, PSO Mean: {pso_data['sharpe_ratio'].mean():.4f}")
#     print("The difference in fitness is statistically significant." if p_value < alpha else "The difference in fitness is NOT statistically significant.")

#     # Perform Wilcoxon test for evaluations
#     stat, p_value = wilcoxon(ga_data["n_evaluations"], pso_data["n_evaluations"])
#     print(f"Evaluations - GA Mean: {ga_data['n_evaluations'].mean():.4f}, PSO Mean: {pso_data['n_evaluations'].mean():.4f}")
#     print("The difference in evaluations is statistically significant." if p_value < alpha else "The difference in evaluations is NOT statistically significant.")


## Friedman Test for Overall Algorithm Comparison

In [ ]:
# from scipy.stats import friedmanchisquare

# # Prepare data for Friedman test
# fitness_data = [
#     combined_results[(combined_results["num_companies"] == num_companies) & (combined_results["algorithm"] == "GA")]["sharpe_ratio"].values
#     for num_companies in combined_results["num_companies"].unique()
# ] + [
#     combined_results[(combined_results["num_companies"] == num_companies) & (combined_results["algorithm"] == "PSO")]["sharpe_ratio"].values
#     for num_companies in combined_results["num_companies"].unique()
# ]

# # Perform Friedman test for fitness
# stat, p_value = friedmanchisquare(*fitness_data)
# print("Friedman Test for Fitness")
# print(f"Statistic: {stat:.4f}, P-Value: {p_value:.4f}")
# print("The difference in fitness across algorithms is statistically significant." if p_value < alpha else "The difference in fitness across algorithms is NOT statistically significant.")

## Fine-Tuning

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wilcoxon, ttest_rel
# Load the final aggregated results
data = pd.read_csv(os.path.join(ROOT_PATH, "experiments","results","final_fine_tuning_results.csv"))

# Set the alpha value for statistical tests
alpha = 0.05

# Statistical Tests
def perform_statistical_tests(ga_data, pso_data, metric, description):
    print(f"\n{description} - Statistical Tests")
    # Drop NaN values to avoid calculation issues
    ga_data = ga_data[metric].dropna()
    pso_data = pso_data[metric].dropna()

    if ga_data.empty or pso_data.empty:
        print("⚠️ One of the data sets is empty after removing NaN values. Skipping test.")
        return

    print(f"GA Mean: {ga_data.mean():.4f}, PSO Mean: {pso_data.mean():.4f}")

    # Wilcoxon test (non-parametric)
    try:
        stat, p_value = wilcoxon(ga_data, pso_data)
        print(f"Wilcoxon Test: Statistic={stat:.4f}, P-Value={p_value:.4f}")
        if p_value < alpha:
            print("Difference is statistically significant.")
        else:
            print("Difference is NOT statistically significant.")
    except ValueError as e:
        print(f"Error in Wilcoxon Test: {e}")

    # Paired t-Test (parametric)
    try:
        stat, p_value = ttest_rel(ga_data, pso_data)
        print(f"Paired t-Test: Statistic={stat:.4f}, P-Value={p_value:.4f}")
        if p_value < alpha:
            print("Difference is statistically significant.")
        else:
            print("Difference is NOT statistically significant.")
    except ValueError as e:
        print(f"Error in Paired t-Test: {e}")

# Perform tests for Best, Median, and Worst Configurations
for quality in ["best", "median", "worst"]:
    ga_data = data[(data["algorithm"] == "GA") & (data["quality"] == quality)]
    pso_data = data[(data["algorithm"] == "PSO") & (data["quality"] == quality)]
    perform_statistical_tests(ga_data, pso_data, "sharpe_ratio", f"Sharpe Ratio ({quality})")
    perform_statistical_tests(ga_data, pso_data, "annual_return", f"Annual Return ({quality})")
    perform_statistical_tests(ga_data, pso_data, "runtime", f"Execution Time ({quality})")

# Visualization Functions
def bar_plot(data, metric, title, ylabel):
    plt.figure(figsize=(8, 6))
    sns.barplot(data=data, x="quality", y=metric, hue="algorithm", errorbar=None)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.show()

def box_plot(data, metric, title, ylabel):
    plt.figure(figsize=(8, 6))
    sns.boxplot(data=data, x="algorithm", y=metric, hue="quality")
    plt.title(title)
    plt.ylabel(ylabel)
    plt.show()

def scatter_plot(data, x_metric, y_metric, title, xlabel, ylabel):
    plt.figure(figsize=(8, 6))
    sns.scatterplot(data=data, x=x_metric, y=y_metric, hue="algorithm", style="quality")
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

# Visualization: Sharpe Ratio Comparison (Best, Median, Worst)
bar_plot(data, "mean_sharpe", "Mean Sharpe Ratios (GA vs PSO)", "Mean Sharpe Ratio")

# Visualization: Return Distribution (Best, Median, Worst)
box_plot(data, "annual_return", "Annual Return Distributions (GA vs PSO)", "Annual Return")

# Visualization: Execution Time Comparison (Best, Median, Worst)
bar_plot(data, "runtime", "Execution Time (GA vs PSO)", "Runtime (seconds)")

# Visualization: Tradeoff Analysis (Sharpe Ratio vs. Time)
scatter_plot(data, "runtime", "sharpe_ratio", "Sharpe Ratio vs. Execution Time (GA vs PSO)", "Execution Time (seconds)", "Sharpe Ratio")

# Best Configuration Comparison (Sharpe Ratio vs Annual Return)
best_data = data[data["quality"] == "best"]
scatter_plot(best_data, "sharpe_ratio", "annual_return", "Best Configuration Comparison (GA vs PSO)", "Sharpe Ratio", "Annual Return")

print("✅ All statistical tests and visualizations completed.")